In [1]:
import os
import pandas as pd
import json
import traceback
from dotenv import load_dotenv
load_dotenv() #take environment variables from .env.

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_community.callbacks.manager import get_openai_callback
import PyPDF2

In [3]:
from langchain_openai import ChatOpenAI
KEY = os.getenv("apikey")


In [4]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [6]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use lit as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json} 
"""

In [7]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [8]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

e:\ALL PROGRAMS\All_DS_Projects\Generative AI\MCQ_Gen\mcqenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
    You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
    if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [10]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=[ "subject",  "quiz"],
    template=TEMPLATE2
    )

In [11]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [12]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [13]:
file_path=r"E:\ALL PROGRAMS\All_DS_Projects\Generative AI\MCQ_Gen\data.txt"

In [14]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [15]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, 


In [16]:
# Serialize the Python dictionary into a JSON-formatted string
print(json.dumps(RESPONSE_JSON))

{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}


In [17]:
NUMBER=3 
SUBJECT="Machine Learning"
TONE="simple"


In [18]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

e:\ALL PROGRAMS\All_DS_Projects\Generative AI\MCQ_Gen\mcqenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, 
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of 3 multiple choice questions for Machine Learning students in simple tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use lit as a guide. Ensure to make 3 MCQs
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice 

In [19]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1010
Prompt Tokens:700
Completion Tokens:310
Total Cost:0.00167


In [20]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nML finds application in many fields, including natural language processing, ',
 'number': 3,
 'subject': 'Machine Learning',
 'tone': 'simple',
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "c

In [24]:
quiz=response.get("quiz")
print(quiz)


{
    "1": {
        "mcq": "What is machine learning?",
        "options": {
            "a": "A field of study in artificial intelligence concerned with developing and studying statistical algorithms.",
            "b": "A field of study in natural language processing.",
            "c": "A field of study in robotics.",
            "d": "A field of study in computer vision."
        },
        "correct": "a"
    },
    "2": {
        "mcq": "What is the main goal of machine learning?",
        "options": {
            "a": "To perform tasks with explicit instructions.",
            "b": "To generalize to unseen data and perform tasks without explicit instructions.",
            "c": "To ignore data and make random decisions.",
            "d": "To only learn from data that has been seen before."
        },
        "correct": "b"
    },
    "3": {
        "mcq": "Which approach has recently surpassed many previous methods in machine learning performance?",
        "options": {
      

In [22]:
quiz = json.loads(quiz)
quiz

{'1': {'mcq': 'What is machine learning?',
  'options': {'a': 'A field of study in artificial intelligence concerned with developing and studying statistical algorithms.',
   'b': 'A field of study in natural language processing.',
   'c': 'A field of study in robotics.',
   'd': 'A field of study in computer vision.'},
  'correct': 'a'},
 '2': {'mcq': 'What is the main goal of machine learning?',
  'options': {'a': 'To perform tasks with explicit instructions.',
   'b': 'To generalize to unseen data and perform tasks without explicit instructions.',
   'c': 'To ignore data and make random decisions.',
   'd': 'To only learn from data that has been seen before.'},
  'correct': 'b'},
 '3': {'mcq': 'Which approach has recently surpassed many previous methods in machine learning performance?',
  'options': {'a': 'Decision trees',
   'b': 'Support vector machines',
   'c': 'Artificial neural networks',
   'd': 'Random forests'},
  'correct': 'c'}}

In [29]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
def get_table_data(quiz_str):
    try:
        quiz_dict=json.loads(quiz_str)
        quiz_table_data=[]
        
        # iterate over the quiz dictionary and extract the required information
        for key,value in quiz_dict.items():
            mcq=value["mcq"]
            options=" || ".join(
                [
                    f"{option}-> {option_value}" for option, option_value in value["options"].items()
                 
                 ]
            )
            
            correct=value["correct"]
            quiz_table_data.append({"MCQ": mcq,"Choices": options, "Correct": correct})

        return quiz_table_data
    
    except Exception as e:        
        raise (e) 

In [30]:
quiz_table_data

[{'MCQ': 'What is machine learning focused on?',
  'Choices': 'a: Developing and studying statistical algorithms | b: Creating explicit instructions for tasks | c: Exploring data analysis through unsupervised learning | d: Applying mathematical optimization methods',
  'Correct': 'a'},
 {'MCQ': 'Which field has been able to surpass many previous approaches in performance?',
  'Choices': 'a: Artificial intelligence | b: Data mining | c: Artificial neural networks | d: Predictive analytics',
  'Correct': 'c'},
 {'MCQ': 'In which field does machine learning find applications?',
  'Choices': 'a: Music composition | b: Agriculture | c: Sports coaching | d: Fashion design',
  'Correct': 'b'},
 {'MCQ': 'What is the related field of study focusing on exploratory data analysis?',
  'Choices': 'a: Machine learning | b: Natural language processing | c: Data mining | d: Speech recognition',
  'Correct': 'c'},
 {'MCQ': 'Which framework provides a theoretical viewpoint for describing machine learnin

In [31]:
quiz=pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,What is machine learning focused on?,a: Developing and studying statistical algorit...,a
1,Which field has been able to surpass many prev...,a: Artificial intelligence | b: Data mining | ...,c
2,In which field does machine learning find appl...,a: Music composition | b: Agriculture | c: Spo...,b
3,What is the related field of study focusing on...,a: Machine learning | b: Natural language proc...,c
4,Which framework provides a theoretical viewpoi...,a: Statistical analysis | b: Probably approxim...,b


In [32]:
quiz.to_csv("machinelearning.csv",index=False)
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_10_2024_14_55_48'